In [1]:
import pandas as pd
import sqlalchemy 
from sqlalchemy import text
from datetime import datetime, timedelta
import os
import sys

# Adicionando caminhos dos módulos
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros

SCHEMA_DEFAULT = "dbo"
CAMINHO_RELATORIOS = Parametros.caminho_completo_atual()

In [2]:
if not CAMINHO_RELATORIOS:
    print("Erro: Pasta de relatórios de hoje não encontrada.")
    sys.exit()

In [3]:
# Definição de datas
data_hoje_str = datetime.today().strftime("%d-%m-%Y")
# Nota: O código original usava dias=7 na variável chamada 'quinze_dias'. Mantive a lógica original.
data_corte = (datetime.today() - timedelta(days=7)).strftime("%Y-%m-%d")

#### CAPTAÇÃO

In [4]:
try:
    # Busca dinâmica do arquivo nnm.xlsx
    caminho_nnm = None
    pastas_busca = [CAMINHO_RELATORIOS, os.path.dirname(CAMINHO_RELATORIOS)]
    
    for pasta in pastas_busca:
        if os.path.exists(os.path.join(pasta, "nnm.xlsx")):
            caminho_nnm = os.path.join(pasta, "nnm.xlsx")
            break
            
    if not caminho_nnm:
        raise FileNotFoundError("Arquivo 'nnm.xlsx' não encontrado.")

    captacao = pd.read_excel(caminho_nnm, header=2)
    
    # Tratamento
    for coluna in captacao.columns:
        captacao.rename(columns={coluna:coluna.upper()}, inplace=True)
        
    captacao['CONTA'] = captacao['CONTA'].astype(str).str.strip()
    captacao.rename(columns={"ASSESSOR":'Assessor'}, inplace=True)
    
    # Ajustes de nomes
    captacao.loc[captacao['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "Rodrigo de Mello D’Elia"
    captacao['Assessor'] = captacao['Assessor'].apply(lambda nome: str(nome).upper())
    
    captacao = captacao.loc[:,["CONTA", "MERCADO", "DESCRIÇÃO", "ATIVO",
                               "CAPTAÇÃO", "DATA", "TIPO", "Assessor"]]
except Exception as e:
    print(f"Erro ao ler NNM: {e}")
    sys.exit()

##### OFFSHORE

In [5]:
try:
    contas_offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
    contas_offshore = contas_offshore.loc[:,["Conta", "Assessor"]]
    
    offshore = pd.read_excel(r"C:\Scripts\historico_nnm\NNM Offshore.xlsx", sheet_name="NNM Offshore")
    offshore['Conta'] = offshore['Conta'].astype(str).str.strip()
    
    offshore = offshore.merge(contas_offshore, on='Conta', how='left')
    offshore = offshore[~offshore['Conta'].isnull()]
    
    offshore.rename(columns={"Data NNM":"DATA", "Conta":"CONTA", "NNM BRL":"CAPTAÇÃO"}, inplace=True)
    if "NNM USD" in offshore.columns:
        offshore.drop(["NNM USD"], axis=1, inplace=True)
        
    offshore['Assessor'] = [str(nome).upper() for nome in offshore['Assessor']]
    offshore['DATA'] = pd.to_datetime(offshore['DATA'], format="%d/%m/%Y")
    offshore['DESCRIÇÃO'] = "OFFSHORE"
    
    # Concatenação
    captacao = pd.concat([offshore, captacao])
    captacao['Assessor'] = captacao['Assessor'].astype(str).apply(lambda nome: nome.upper())

except Exception as e:
    print(f"Erro em Offshore: {e}")

##### MIGRAÇÕES

In [6]:
try:
    # Leitura do banco Azure
    migracoes_btg = AzureLoader.ler_tabela("migracoes_btg", schema=SCHEMA_DEFAULT)
    
    # Leitura do arquivo de troca de assessor
    caminho_troca = os.path.join(os.path.dirname(CAMINHO_RELATORIOS), "troca_assessor.xlsx")
    # Tenta na pasta do dia se não achar na raiz da data
    if not os.path.exists(caminho_troca):
         caminho_troca = os.path.join(CAMINHO_RELATORIOS, "troca_assessor.xlsx")

    troca_assessores = pd.read_excel(caminho_troca)
    
    # Filtros de contas específicas e troca de assessor
    contas_filtro = [
        '590732', '299305', '5173757', '5152837', 
        '5149832', '5917705', '15296593'
    ]
    
    migracoes_btg['CONTA'] = migracoes_btg['CONTA'].astype(str).str.strip()
    
    migracoes_btg = migracoes_btg[
        (migracoes_btg['CONTA'].isin(troca_assessores['Conta'].astype(str))) |
        (migracoes_btg['CONTA'].isin(contas_filtro))
    ]
    
    captacao['TIPO DE CAPTACAO'] = "Padrão"
    migracoes_btg['TIPO DE CAPTACAO'] = "Migração BTG"
    migracoes_btg['DESCRIÇÃO'] = "Migração BTG"
    migracoes_btg['MERCADO'] = "Migração BTG"
    
    # Consolidação
    captacao = pd.concat([captacao, migracoes_btg], axis=0)
    captacao = captacao[captacao['TIPO'] != "RS"]
    captacao.dropna(subset='DATA', axis=0, inplace=True)
    
    captacao['CONTA'] = captacao['CONTA'].astype(str)
    captacao['CAPTAÇÃO'] = captacao['CAPTAÇÃO'].astype(float)
    captacao['DATA'] = pd.to_datetime(captacao['DATA'], errors='coerce') # Ajustado para evitar erros de formato
    
    captacao = captacao[['DATA', 'CONTA', 'CAPTAÇÃO', 'Assessor', 'TIPO DE CAPTACAO', 'MERCADO']]
    
    # Filtro de data recente (os últimos dias definidos)
    captacao = captacao[captacao['DATA'] >= data_corte].sort_values(by='DATA', ascending=False)

except Exception as e:
    print(f"Erro em Migrações: {e}")

[AzureLoader] Lendo: dbo.migracoes_btg...
Erro em Migrações: [Errno 2] No such file or directory: 'C:\\Scripts\\relatórios\\\\06-02-2026\\0848\\troca_assessor.xlsx'


##### CAPTAÇÃO HISTORICO

In [7]:
try:
    # Lê histórico do Azure
    captacao_historico = AzureLoader.ler_tabela("captacao_historico", schema=SCHEMA_DEFAULT)
    
    # Remove período que será reprocessado (substituição)
    captacao_historico = captacao_historico[captacao_historico['DATA'] < data_corte]
    
    # Lê tabelas auxiliares
    primeiro_vinculo = AzureLoader.ler_tabela("primeiro_vinculo", schema=SCHEMA_DEFAULT)
    primeiro_vinculo.rename(columns={"Conta":"CONTA"}, inplace=True)
    
    # Lê Base BTG do Azure (Substitui Bases.basebtg())
    basebtg = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
    
    # Concatena novo período processado com histórico antigo
    captacao_historico = pd.concat([captacao, captacao_historico], axis=0)
    
    # Identifica Situação (Ativo/Inativo)
    lista_contas_ativas = set(basebtg['Conta'].astype(str))
    captacao_historico['CONTA'] = captacao_historico['CONTA'].astype(str).str.strip()
    
    # Otimização com map/apply é mais rápida que loop for
    captacao_historico['Situacao'] = captacao_historico['CONTA'].apply(
        lambda x: "Ativo" if x in lista_contas_ativas else "Inativo"
    )

    # --- Lógica de Saída de Conta (Churn) ---
    # Lê histórico diário de PL via SQL Query específica
    db = Connect()
    conn = db.connect_techdb()
    query_pl = f'SELECT "Conta", "PL Total", "Data" FROM {SCHEMA_DEFAULT}."pl_historico_diario"'
    pl_historico = pd.read_sql(query_pl, conn)
    conn.close()
    
    pl_historico.rename(columns={"Data":"DATA", "Conta":"CONTA"}, inplace=True)
    pl_historico['CONTA'] = pl_historico['CONTA'].astype(str).str.strip()
    pl_historico['DATA'] = pd.to_datetime(pl_historico['DATA'])
    pl_historico.sort_values("DATA", inplace=True)
    
    contas_inativas = captacao_historico[captacao_historico['Situacao'] == "Inativo"].drop_duplicates(subset='CONTA')
    
    debitos = []
    
    # Loop de cálculo de churn
    for _, row in contas_inativas.iterrows():
        conta = row['CONTA']
        assessor = row['Assessor']
        situacao = row['Situacao']
        
        hist_conta = pl_historico[pl_historico['CONTA'] == conta]
        
        if not hist_conta.empty:
            # Pega último PL e inverte sinal
            valor_captacao = hist_conta.iloc[-1]['PL Total'] * -1
            data_evento = hist_conta['DATA'].max()
        else:
            valor_captacao = 0
            data_evento = pd.to_datetime(data_corte) # Fallback

        debitos.append({
            "DATA": data_evento,
            "CONTA": conta,
            "CAPTAÇÃO": valor_captacao,
            "Assessor": assessor,
            "Situacao": situacao
        })
    
    if debitos:
        debitos_df = pd.DataFrame(debitos)
        debitos_df = debitos_df.drop_duplicates(subset='CONTA', keep='first')
        debitos_df = debitos_df[debitos_df['DATA'] >= data_corte]
        
        # Cruzamento com Entradas e Saídas
        entradas_e_saidas = AzureLoader.ler_tabela("Entradas_e_saidas_consolidado", schema=SCHEMA_DEFAULT)
        entradas_e_saidas = entradas_e_saidas[['Conta', 'Mês de entrada/saída']].copy()
        entradas_e_saidas.rename(columns={"Conta":"CONTA"}, inplace=True)
        entradas_e_saidas['CONTA'] = entradas_e_saidas['CONTA'].astype(str).str.strip()
        entradas_e_saidas = entradas_e_saidas.drop_duplicates("CONTA", keep='last')
        
        debitos_df = pd.merge(debitos_df, entradas_e_saidas, on='CONTA', how='left')
        
        # Ajuste de Datas
        debitos_df['Mês de entrada/saída'] = pd.to_datetime(debitos_df['Mês de entrada/saída'], format="%d-%m-%Y", errors='coerce')
        debitos_df['DATA'] = debitos_df['Mês de entrada/saída'].fillna(debitos_df['DATA'])
        
        debitos_df.drop('Mês de entrada/saída', axis=1, inplace=True)
        debitos_df = debitos_df.dropna(subset=['DATA'])
        
        debitos_df['TIPO DE CAPTACAO'] = "Saída de conta"
        debitos_df['MERCADO'] = "Saída de conta"
        
        captacao_historico = pd.concat([captacao_historico, debitos_df], axis=0)

except Exception as e:
    print(f"Erro em Histórico/Churn: {e}")

[AzureLoader] Lendo: dbo.captacao_historico...
[AzureLoader] Lendo: dbo.primeiro_vinculo...
[AzureLoader] Lendo: dbo.base_btg...
[AzureLoader] Lendo: dbo.Entradas_e_saidas_consolidado...


In [ ]:
try:
    # Atualizar Assessor Atual usando base_btg
    assessor_atual = basebtg[['Conta', 'Assessor']].copy()
    assessor_atual.rename(columns={"Conta":"CONTA", "Assessor":"Assessor atual"}, inplace=True)
    assessor_atual['CONTA'] = assessor_atual['CONTA'].astype(str).str.strip()
    
    captacao_historico = captacao_historico.merge(assessor_atual, on='CONTA', how='left')
    
    captacao_historico['Assessor atual'] = captacao_historico['Assessor atual'].fillna(captacao_historico['Assessor'])
    captacao_historico.drop("Assessor", axis=1, inplace=True)
    captacao_historico.rename(columns={"Assessor atual":"Assessor"}, inplace=True)
    
    captacao_historico['DATA'] = pd.to_datetime(captacao_historico['DATA'])
    
    # Merge Nomes
    nomes_clientes = pd.read_excel(r"C:\\Scripts\\nomes_clientes\Nomes_Clientes2.xlsx")
    nomes_clientes.rename(columns={"Conta":"CONTA"}, inplace=True)
    nomes_clientes['CONTA'] = nomes_clientes['CONTA'].astype(str).str.strip()
    nomes_clientes.drop_duplicates("CONTA", inplace=True)
    
    if "Nome" in captacao_historico.columns:
        captacao_historico.drop("Nome", axis=1, inplace=True)
        
    captacao_historico = captacao_historico.merge(nomes_clientes, on='CONTA', how='left')

except Exception as e:
    print(f"Erro nos tratamentos finais: {e}")

In [10]:
try:
    # 1. Padronizar todas as colunas para MAIÚSCULO
    # Isso resolve o conflito 'Tipo' vs 'TIPO' (ambos viram TIPO)
    captacao_historico.columns = captacao_historico.columns.str.upper()

    # 2. Remover colunas duplicadas
    # Se existiam dois 'TIPO', esse comando remove o segundo e mantém o primeiro
    captacao_historico = captacao_historico.loc[:, ~captacao_historico.columns.duplicated()]

    # 3. Tratamento de Conflito: SITUACAO vs SITUAÇÃO (acentuação)
    if 'SITUAÇÃO' in captacao_historico.columns:
        if 'SITUACAO' in captacao_historico.columns:
            # Se as duas existem, preenche a sem acento e apaga a com acento
            captacao_historico['SITUACAO'] = captacao_historico['SITUACAO'].fillna(captacao_historico['SITUAÇÃO'])
            captacao_historico.drop('SITUAÇÃO', axis=1, inplace=True)
        else:
            # Se só existe a com acento, renomeia
            captacao_historico.rename(columns={'SITUAÇÃO': 'SITUACAO'}, inplace=True)

    # 4. Garantir tipos de dados
    captacao_historico['CONTA'] = captacao_historico['CONTA'].astype(str).str.strip()
    
    # 5. Envio Seguro
    AzureLoader.enviar_df(
        df=captacao_historico,
        nome_tabela="captacao_historico",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )

    print("--- Processo de Captação Finalizado com Sucesso ---")

except Exception as e:
    print(f"Erro no Upload Final: {e}")

C:\Users\joaob\AppData\Local\Temp\ipykernel_26164\1528210914.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captacao_historico['SITUACAO'] = captacao_historico['SITUACAO'].fillna(captacao_historico['SITUAÇÃO'])
C:\Users\joaob\AppData\Local\Temp\ipykernel_26164\1528210914.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  captacao_historico.drop('SITUAÇÃO', axis=1, inplace=True)
C:\Users\joaob\AppData\Local\Temp\ipykernel_26164\1528210914.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

[AzureLoader] Subindo tabela: dbo.captacao_historico (69140 linhas)...
[AzureLoader] Chunksize calculado: 190 linhas/lote (Colunas: 11).
[AzureLoader] Concluido: captacao_historico atualizada.
--- Processo de Captação Finalizado com Sucesso ---
